In [1]:
from elasticsearch import Elasticsearch

In [2]:
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic","QGm3UkTIuPLi*PAdmWjR"),
    ca_certs="C:/Users/Geraldus Wilsen/Downloads/elasticsearch-8.11.1/config/certs/http_ca.crt"
)
es.ping()

True

In [12]:
import pandas as pd

# df = pd.read_csv("data_fix_obat_with_nan_value.csv", sep = ";")
df = pd.read_csv("data_obat.csv", usecols=["id", "name", "uses"], sep = ";").head(500)
df.fillna("None", inplace=True)
df.head()

,id,name,uses
0,0,Sildenafil Citrate 50 mg 4 Tablet,SILDENAFIL CITRATE merupakan obat yang digunak...
1,1,Viagra 50 mg 1 Tablet,VIAGRA merupakan obat yang digunakan untuk men...
2,2,Ericfil Tablet 100 mg,ERICFIL merupakan obat yang digunakan untuk me...
3,3,Cialis Tablet 10 mg,CIALIS 10 MG merupakan obat dengan kandungan T...
4,4,Ericfil 50 mg Odf,ERICFIL 50 MG ODF merupakan obat yang digunaka...


In [13]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

c:\Users\Geraldus Wilsen\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
es.ping()

True

In [16]:
df["uses_vector"] = df["uses"].apply(lambda x: model.encode(x))

In [17]:
df.head()

,id,name,uses,uses_vector
0,0,Sildenafil Citrate 50 mg 4 Tablet,SILDENAFIL CITRATE merupakan obat yang digunak...,"[0.009720819, -0.03853779, 0.0065294574, -0.02..."
1,1,Viagra 50 mg 1 Tablet,VIAGRA merupakan obat yang digunakan untuk men...,"[0.017517975, -0.047177713, 0.013666437, -0.01..."
2,2,Ericfil Tablet 100 mg,ERICFIL merupakan obat yang digunakan untuk me...,"[0.052561115, -0.06175869, 0.026492704, -0.014..."
3,3,Cialis Tablet 10 mg,CIALIS 10 MG merupakan obat dengan kandungan T...,"[0.0005132847, -0.056769144, -0.00086643675, -..."
4,4,Ericfil 50 mg Odf,ERICFIL 50 MG ODF merupakan obat yang digunaka...,"[0.06265341, -0.06602495, 0.023286238, -0.0229..."


In [19]:
len(df['uses_vector'][0])

768

In [20]:
from indexMapping import indexMapping

es.indices.create(index="data_obat", mappings=indexMapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'data_obat'})

In [21]:
record_list = df.to_dict("records")

In [22]:
for record in record_list:
    try:
        es.index(index="data_obat", document=record, id=record["id"])
    except Exception as e:
        print(e)

In [23]:
es.count(index="data_obat")

ObjectApiResponse({'count': 500, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

In [24]:
input_keyword = "obat sakit kepala"
vector_of_input_keyword = model.encode(input_keyword)

query = {
    "field" : "uses_vector",
    "query_vector" : vector_of_input_keyword,
    "k" : 10,
    "num_candidates" : 500, 
}

res = es.knn_search(index="data_obat", knn=query , source=["name","uses"])
res["hits"]["hits"]

C:\Users\Geraldus Wilsen\AppData\Local\Temp\ipykernel_17604\695743240.py:11: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index="data_obat", knn=query , source=["name","uses"])


[{'_index': 'data_obat',
  '_id': '25',
  '_score': 0.6034966,
  '_source': {'name': 'Viajoy Kaplet 100 mg',
   'uses': 'VIAJOY merupakan obat dengan kandungan Sildenafil dalam bentuk kaplet salut selaput. Obat ini digunakan untuk disfungsi ereksi. Dalam penggunaan obat ini harus SESUAI DENGAN PETUNJUK DOKTER.'}},
 {'_index': 'data_obat',
  '_id': '88',
  '_score': 0.6034966,
  '_source': {'name': 'Viajoy Kaplet 100 mg',
   'uses': 'VIAJOY merupakan obat dengan kandungan Sildenafil dalam bentuk kaplet salut selaput. Obat ini digunakan untuk disfungsi ereksi. Dalam penggunaan obat ini harus SESUAI DENGAN PETUNJUK DOKTER.'}},
 {'_index': 'data_obat',
  '_id': '10',
  '_score': 0.5925721,
  '_source': {'name': 'Topgra 100 mg 1 Tablet',
   'uses': 'TOPGRA merupakan obat yang digunakan untuk mengobati gangguan fungsi seksual pada pria, yaitu disfungsi ereksi. Obat ini bekerja dengan cara meningkatkan aliran darah ke alat kelamin pria sehingga dapat mempertahankan ereksi saat berhubungan sek